# Neural network learning from scratch (Math + Numpy)

This is a major milestone during my self-taught machine learning journey. Understanding the backpropagation and calculus in it is quite an experience. That said, this is really the simplest implementation :p.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Normalization
from tensorflow.keras.activations import linear, relu, sigmoid, softmax
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from keras.datasets import mnist

### The dataset(diagnosing malignancy given texture_mean and radius_mean)

In [4]:
df = pd.read_csv("bdiag.csv")[["diagnosis", "texture_mean", "radius_mean"]]
df["diagnosis"] = df["diagnosis"].replace({"M":1, "B":0})
df

,diagnosis,texture_mean,radius_mean
0,1,10.38,17.99
1,1,17.77,20.57
2,1,21.25,19.69
3,1,20.38,11.42
4,1,14.34,20.29
...,...,...,...
564,1,22.39,21.56
565,1,28.25,20.13
566,1,28.08,16.60
567,1,29.33,20.60


### Train-test split

In [6]:
X = df[["texture_mean", "radius_mean"]].values
y = df["diagnosis"].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [9]:
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(y_test.shape))

X_train: (455, 2)
Y_train: (455, 1)
X_test:  (114, 2)
Y_test:  (114, 1)


### Feature normalization

In [7]:
norm = Normalization(axis=1)
norm.adapt(X_train)

In [10]:
X_train = np.array(norm(X_train))  # converts back to numpy because tensor is slower
X_test = np.array(norm(X_test))

### The Model overview

![alt text](model_nn_scratch.jpg "Title")

## Doing it the "easy" way (using library)

In [19]:
model = Sequential([
    
    Dense(units=2, activation=relu),
    Dense(units=1, activation=linear)
])

model.compile(loss=BinaryCrossentropy(from_logits=True),
              optimizer=Adam(learning_rate=0.001),
              metrics="accuracy"
             )
model.fit(X_train, y_train, epochs=30)

Epoch 1/30
15/15 [==============================] - 3s 8ms/step - loss: 0.5537 - accuracy: 0.8527
Epoch 2/30
15/15 [==============================] - 0s 3ms/step - loss: 0.5467 - accuracy: 0.8527
Epoch 3/30
15/15 [==============================] - 0s 3ms/step - loss: 0.5397 - accuracy: 0.8593
Epoch 4/30
15/15 [==============================] - 0s 4ms/step - loss: 0.5328 - accuracy: 0.8615
Epoch 5/30
15/15 [==============================] - 0s 5ms/step - loss: 0.5261 - accuracy: 0.8659
Epoch 6/30
15/15 [==============================] - 0s 3ms/step - loss: 0.5193 - accuracy: 0.8659
Epoch 7/30
15/15 [==============================] - 0s 5ms/step - loss: 0.5126 - accuracy: 0.8681
Epoch 8/30
15/15 [==============================] - 0s 3ms/step - loss: 0.5061 - accuracy: 0.8681
Epoch 9/30
15/15 [==============================] - 0s 3ms/step - loss: 0.4995 - accuracy: 0.8681
Epoch 10/30
15/15 [==============================] - 0s 3ms/step - loss: 0.4929 - accuracy: 0.8681
Epoch 11/30
15/15 [

### Evaluation

In [20]:
model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 5ms/step - loss: 0.3928 - accuracy: 0.8947


[0.3927581310272217, 0.8947368264198303]

# Doing it the hard way (scratch)

### The math

2 features --> 1 hidden layer (2 neuron relu) --> 1 output layer (1 neuron sigmoid)

#### Forward propagation:

$$z_{1}^{[1]} = w_{1}^{[1]}x_{1} + w_{2}^{[1]}x_{2} + b_{1}^{[1]} $$ <br>
$$z_{2}^{[1]} = w_{3}^{[1]}x_{1} + w_{4}^{[1]}x_{2} + b_{2}^{[1]} $$ <br>
$$a_{1}^{[1]} = \text{relu}(z_{1}^{[1]})$$ <br>
$$a_{2}^{[1]} = \text{relu}(z_{2}^{[1]})$$ <br>
$$z^{[2]} = w_{1}^{[2]}x_{1} + w_{2}^{[2]}x_{2} + b^{[2]} $$ <br>
$$a^{[2]} = \frac{1}{1 + e^{-z^{[2]}}}$$ <br>

#### cost & loss function: <br>
$$\text{cross entropy cost}(\hat{a}^{[2]}, \hat{y}) = -\frac{1}{n}\sum_{i=1}^{n}\sum_{j=1}^{m}y_{ij}\text{log}(a^{[2]}[ij])$$ <br>
$$\text{binary cross entropy cost}(\hat{a}^{[2]}, \hat{y}) = -\frac{1}{n}\sum_{i=1}^{n}y_{i}\text{log}(a^{[2]}[i]) + (1-y_{i})\text{log}(1-a^{[2]}[i])$$ <br>

$$\frac{\partial ( \text{binary cross entropy cost}(\hat{a}^{[2]}, \hat{y}))}{\partial \vec{w}} = \frac{1}{n}(\frac {\partial (\text{binary cross entropy loss}(a^{[2]}, \hat{y}))_{1}}{\partial \vec{w}} + \frac {\partial (\text{binary cross entropy loss}(a^{[2]}, \hat{y}))_{2}}{\partial \vec{w}}+ ...+ \frac {\partial (\text{binary cross entropy loss}(a^{[2]}, \hat{y}))_{n}}{\partial \vec{w}})\space \text{for n=number of examples}$$<br>

$$\text{binary cross entropy loss}(a^{[2]}, \hat{y}) = -(y_{i}\text{log}(a^{[2]}) + (1-y_{i})\text{log}(1-a^{[2]}))$$

- The point of backpropagation is to determine the gradient of cost function wrt to each parameter $w$
- Intuitively speaking, the gradient of cost function wrt to parameter $w$ is the average of all the gradient of loss function wrt to parameter $w_{1}, w_{2},..., w_{n}$ where $n=$number of examples.
- Backpropagation involves the derivation of loss function wrt to each parameters, then averaging them later for gradient descent

#### Backward propagation:

##### first layer
$\frac{\partial \text{cost}(a^{[2]})}{\partial z^{[2]}}=\frac{\partial a^{[2]}}{\partial z^{[2]}}\times\frac{\partial\text{cost}(a^{[2]})}{\partial a^{[2]}} = (1-a^{[2]})\times \frac{a^{[2]}-y}{a^{[2]}(1-a^{[2]})} = a^{[2]} - y$


$$\frac{\partial\text{cost}(a^{[2]})}{\partial w_{1}^{2}} = \frac{\partial z^{[2]}}{\partial w_{1}^{[2]}}\times\frac{\partial \text{cost}(a^{[2]})}{\partial z^{[2]}} = a_{1}^{[1]}\times (a^{[2]} - y)$$

$$\frac{\partial\text{cost}(a^{[2]})}{\partial w_{2}^{2}} = \frac{\partial z^{[2]}}{\partial w_{2}^{[2]}}\times\frac{\partial \text{cost}(a^{[2]})}{\partial z^{[2]}} = a_{2}^{[1]}\times (a^{[2]} - y)$$
. <br>
.(continue yourself :p) <br>
. <br>

##### Second layer
$\frac{\partial \text{cost}(a^{[2]})}{\partial z_{1}^{[1]}}=\frac{\partial a_{1}^{[1]}}{\partial z_{1}^{[1]}}\times\frac{\partial z^{[2]}}{\partial a_{1}^{[1]}}\times\frac{\partial \text{cost}(a^{[2]})}{\partial z^{[2]}} = \text{relu'}(z_1^{[1]}) \times w_1^{[2]}\times (a^{[2]} - y)$ <br>

$\frac{\partial \text{cost}(a^{[2]})}{\partial z_{2}^{[1]}}=\frac{\partial a_{1}^{[1]}}{\partial z_{2}^{[1]}}\times\frac{\partial z^{[2]}}{\partial a_{1}^{[1]}}\times\frac{\partial \text{cost}(a^{[2]})}{\partial z^{[2]}} = \text{relu'}(z_2^{[1]}) \times w_2^{[2]}\times (a^{[2]} - y)$ <br>

$$\frac{\partial\text{cost}(a^{[2]})}{\partial w_{1}^{1}} = \frac{\partial z_{1}^{[1]}}{\partial w_{1}^{[1]}}\times \frac{\partial \text{cost}(a^{[2]})}{\partial z_{1}^{[1]}} = x_1\times \text{relu'}(z_1^{[1]}) \times w_1^{[2]}\times (a^{[2]} - y)$$ 
. <br>
.(continue yourself :p) <br>
. <br>
$$\frac{\partial\text{cost}(a^{[2]})}{\partial w_{3}^{1}} = \frac{\partial z_{2}^{[1]}}{\partial w_{3}^{[1]}}\times \frac{\partial \text{cost}(a^{[2]})}{\partial z_{1}^{[1]}} = x_2\times \text{relu'}(z_2^{[1]}) \times w_2^{[2]}\times (a^{[2]} - y)$$
. <br>
.(continue yourself :p) <br>
. <br>

### No vectorization and non-modular implementation (Not efficient)

In [23]:
class NeuralNetwork():
    
    def initialize(self):
        self.w11, self.w12, self.w13, self.w14, self.w21, self.w22, self.b11, self.b12, self.b2 = np.random.randn(9)
    
    def relu(self, x):
        return np.maximum(x, 0)
    
    def sigmoid(self, x):
        return 1/(1+np.exp(-x))
    
    def binary_cross_entropy(self, x, y_true):
        return -(y_true * np.log(x) + (1 - y_true) * np.log(1 - x))
    
    def derivative_relu(self, x):
        return 1 if x>0 else 0
    
    def forward_prop(self, x1, x2):
        forward_dict = {} 
        
        forward_dict["z11"] = self.w11 * x1 + self.w12 * x2 + self.b11
        forward_dict["z12"] = self.w13 * x1 + self.w14 * x2 + self.b12
        
        forward_dict["a11"] = self.relu(forward_dict["z11"])
        forward_dict["a12"] = self.relu(forward_dict["z12"])
        
        forward_dict["z2"] = self.w21 * forward_dict["a11"] + self.w22 * forward_dict["a12"] + self.b2
        forward_dict["a2"] = self.sigmoid(forward_dict["z2"])
        
        return forward_dict

    def back_prop(self, x1, x2, y_true, forward_dict):
        #Gradient will be calculated for each training example and averaged later#
        
        deriva_dict = {}
        
        error = forward_dict["a2"] - y_true
        deriva_dict["dloss_dw21"] = forward_dict["a11"] * error
        deriva_dict["dloss_dw22"] = forward_dict["a12"] * error
        deriva_dict["dloss_db2"] = error
        
        dcost_dz11 = self.derivative_relu(forward_dict["z11"]) * self.w21 * error
        dcost_dz12 = self.derivative_relu(forward_dict["z12"]) * self.w22 * error
        
        deriva_dict["dloss_dw11"] = x1 * dcost_dz11
        deriva_dict["dloss_dw12"] = x2 * dcost_dz11
        deriva_dict["dloss_db11"] = dcost_dz11
        
        deriva_dict["dloss_dw13"] = x1 * dcost_dz12
        deriva_dict["dloss_dw14"] = x2 * dcost_dz12
        deriva_dict["dloss_db12"] = dcost_dz12
        
        return deriva_dict
    
    def get_prob_class(self, x1, x2):
        prob = self.forward_prop(x1, x2)["a2"]
        class_ = self.encode_class(prob)
        return prob, class_
    
    def encode_class(self, x):
        return 1 if x>0.5 else 0
    
    def get_accuracy_cost(self, X, y):
        n = X.shape[0]
        n_correct = 0
        cost = 0
        for i in range(n):
            x1, x2 = X[i]
            prediction = self.get_prob_class(x1, x2)
            if prediction[1] == y[i][0]:
                n_correct += 1
            cost += self.binary_cross_entropy(prediction[0], y[i][0])
        return n_correct/n, cost

    def fit(self, X, y, alpha, epochs):
        n, m = X.shape
        
        self.initialize()
        costs = []

        for i in range(1, epochs+1):
            dcost_dw21 = 0
            dcost_dw22 = 0
            dcost_db2 = 0
            dcost_dw11 = 0
            dcost_dw12 = 0
            dcost_db11 = 0
            dcost_dw13 = 0
            dcost_dw14 = 0
            dcost_db12 = 0
            
            for j in range(n):
                x1 , x2 = X[j]
                y_true = y[j][0]
                forward_prop = self.forward_prop(x1, x2)
                back_prop = self.back_prop(x1, x2, y_true, forward_prop)
                
                dcost_dw21 += back_prop["dloss_dw21"]
                dcost_dw22 += back_prop["dloss_dw22"]
                dcost_db2 += back_prop["dloss_db2"]
                dcost_dw11 += back_prop["dloss_dw11"]
                dcost_dw12 += back_prop["dloss_dw12"]
                dcost_db11 += back_prop["dloss_db11"]
                dcost_dw13 += back_prop["dloss_dw13"]
                dcost_dw14 += back_prop["dloss_dw14"]
                dcost_db12 += back_prop["dloss_db12"]
                    
            self.w21 -= alpha*(1/n)*dcost_dw21 
            self.w22 -= alpha*(1/n)*dcost_dw22 
            self.b2  -= alpha*(1/n)*dcost_db2
            self.w11 -= alpha*(1/n)*dcost_dw11 
            self.w12 -= alpha*(1/n)*dcost_dw12 
            self.b11 -= alpha*(1/n)*dcost_db11
            self.w13 -= alpha*(1/n)*dcost_dw13
            self.w14 -= alpha*(1/n)*dcost_dw14
            self.b12 -= alpha*(1/n)*dcost_db12
            
            accuracy, cost = self.get_accuracy_cost(X, y)
            costs.append(cost)
            
            print(f"epoch: {i}, cost: {cost:.2f}, accuracy: {accuracy*100:.2f}%")

In [25]:
classifier = NeuralNetwork()
classifier.fit(X_train, y_train, 0.1, 100)

epoch: 1, cost: 467.80, accuracy: 49.89%
epoch: 2, cost: 420.67, accuracy: 53.63%
epoch: 3, cost: 383.45, accuracy: 57.36%
epoch: 4, cost: 353.63, accuracy: 60.66%
epoch: 5, cost: 329.25, accuracy: 63.52%
epoch: 6, cost: 309.02, accuracy: 68.13%
epoch: 7, cost: 292.33, accuracy: 70.55%
epoch: 8, cost: 278.24, accuracy: 73.85%
epoch: 9, cost: 266.12, accuracy: 75.60%
epoch: 10, cost: 255.51, accuracy: 78.24%
epoch: 11, cost: 246.21, accuracy: 78.68%
epoch: 12, cost: 238.06, accuracy: 79.78%
epoch: 13, cost: 230.94, accuracy: 80.66%
epoch: 14, cost: 224.59, accuracy: 81.54%
epoch: 15, cost: 218.89, accuracy: 81.98%
epoch: 16, cost: 213.69, accuracy: 82.86%
epoch: 17, cost: 209.02, accuracy: 83.08%
epoch: 18, cost: 204.74, accuracy: 83.96%
epoch: 19, cost: 200.83, accuracy: 84.62%
epoch: 20, cost: 197.25, accuracy: 85.05%
epoch: 21, cost: 193.94, accuracy: 85.71%
epoch: 22, cost: 190.91, accuracy: 86.15%
epoch: 23, cost: 188.09, accuracy: 85.71%
epoch: 24, cost: 185.44, accuracy: 85.93%
e

### Evaluation

In [26]:
classifier.get_accuracy_cost(X_test, y_test)

(0.9035087719298246, 32.74126953834825)

#### not bad at all :p